In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
sys.path.append(project_root)

print(project_root)

import machine_learning.ml_lib as ML
import machine_learning.visualisation_lib as VL
import machine_learning.files_lib as FL
from datetime import datetime
from data_extractor.data_extractor import load_df, extract_X_y_from_df 

/Users/gsk/documents/projects/Lie-Detector/classificators_and_data


In [7]:
df = load_df("../../../data/")
# Filter the data and add the column 'label' depends what you need
df = df.query("desired_answer == answer and data_type in ['REAL', 'FAKE']")
df['label'] = df.apply(lambda x: 1 if x.block_no in [1,3] else 0, axis = 1)
df.head()

Opening raw data file ../../../data/1299BF1A/EEG_ExperimentBlock.HONEST_RESPONSE_TO_TRUE_IDENTITY_raw.fif...


    Range : 0 ... 70174 =      0.000 ...   280.696 secs
Ready.
Reading 0 ... 70174  =      0.000 ...   280.696 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 825 samples (3.300 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window wi

,subject,block_no,duration,field,data_type,answer,eeg,desired_answer,label
1,1299BF1A,1,0.840,BIRTH_DATE,REAL,YES,"<Epochs | 1 events (good & bad), 0 – 1 s (base...",YES,1
2,1299BF1A,1,0.744,HOMETOWN,REAL,YES,"<Epochs | 1 events (good & bad), 0 – 1 s (base...",YES,1
3,1299BF1A,1,0.676,HOMETOWN,REAL,YES,"<Epochs | 1 events (good & bad), 0 – 1 s (base...",YES,1
4,1299BF1A,1,0.620,HOMETOWN,REAL,YES,"<Epochs | 1 events (good & bad), 0 – 1 s (base...",YES,1
6,1299BF1A,1,0.652,NAME,REAL,YES,"<Epochs | 1 events (good & bad), 0 – 1 s (base...",YES,1


In [ ]:
X,y = extract_X_y_from_df(df)

In [5]:
# TODO play with frequencies and bands with data and manually check what wasn't checked in below's param grid. Check 0.5 to 1. Check if theres a tendency in learning.

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__max_features': ['sqrt', None],
    'classifier__bootstrap': [True, False],
    'classifier__class_weight': [None, 'balanced']
}

result_folder = "../results/random_forest/"


In [6]:
best_model, best_params, test_score = ML.grid_search_random_forest_eeg(
    X, y, param_grid, test_size=0.2, cv=5, scoring='accuracy'
)

In [ ]:
f"{datetime.now().strftime('%Y-%m-%d_%H:%M:%S_%f')}"

In [ ]:
# file_path = f"../results/random_forest_eeg_20240930_224506/result_RandomForestClassifier_grid_search_random_forest_eeg_5fold_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S_%f')}.json"
file_path = "/Users/gsk/documents/neuron/lie-detector-ai/gs/results/random_forest_eeg_20240930_224506/result_RandomForestClassifier_grid_search_random_forest_eeg_5fold_1727733509.json"
cv_results_df, best_params, best_score, test_score, classification_report = FL.read_grid_search_results(file_path)

print("Best Parameters:")
print(best_params)
print(f"\nBest Cross-Validation Score: {best_score:.4f}")
print(f"Test Score: {test_score:.4f}\n")

VL.plot_mean_test_scores(cv_results_df, 'classifier__max_depth')
VL.plot_mean_test_scores(cv_results_df, 'classifier__n_estimators')
VL.plot_mean_test_scores(cv_results_df, 'classifier__min_samples_split')

VL.plot_heatmap_mean_test_scores(cv_results_df, 'classifier__n_estimators', 'classifier__max_depth')

# VL.plot_fit_and_score_times(cv_results_df, 'classifier__n_estimators')

if classification_report:
    print("Classification Report:")
    for label, metrics in classification_report.items():
        if isinstance(metrics, dict):
            if label in ['accuracy', 'macro avg', 'weighted avg']:
                print(f"\n{label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name}: {metric_value:.4f}")
            else:
                print(f"\nClass {label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name}: {metric_value:.4f}")


In [9]:
param_grid = {
    'classifier__n_estimators': [300],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__max_features': ['sqrt', None],
    'classifier__bootstrap': [True, False],
    'classifier__class_weight': [None, 'balanced']
}

result_folder = "../results/random_forest_test/"


In [ ]:
best_model2, best_params2, test_score2 = ML.grid_search_random_forest_eeg(
    X, y, param_grid, test_size=0.2, cv=5, scoring='accuracy'
)

In [5]:
# I checked few parameters before and I wanted to see if tendencies we assumed from result analasys will give us better results
param_grid_3 = {
    'classifier__n_estimators': [400],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__max_features': ['sqrt', None],
    'classifier__bootstrap': [True, False],
    'classifier__class_weight': [None, 'balanced']
}

result_folder = "../results/random_forest_n_estimators_400/"


In [ ]:
best_model3, best_params3, test_score3 = ML.grid_search_random_forest_eeg(
    X, y, param_grid_3, test_size=0.2, cv=5, scoring='accuracy'
)

In [ ]:
import machine_learning.visualisation_lib as VL


file_path = "your_file_path.json"
cv_results_df, best_params, best_score, test_score, classification_report = FL.read_grid_search_results(file_path)

print("Best Parameters:")
print(best_params)
print(f"\nBest Cross-Validation Score: {best_score:.4f}")
print(f"Test Score: {test_score:.4f}\n")

VL.plot_mean_test_scores(cv_results_df, 'classifier__max_depth')
VL.plot_mean_test_scores(cv_results_df, 'classifier__n_estimators')
VL.plot_mean_test_scores(cv_results_df, 'classifier__min_samples_split')

VL.plot_heatmap_mean_test_scores(cv_results_df, 'classifier__n_estimators', 'classifier__max_depth')

# VL.plot_fit_and_score_times(cv_results_df, 'classifier__n_estimators')

if classification_report:
    print("Classification Report:")
    for label, metrics in classification_report.items():
        if isinstance(metrics, dict):
            if label in ['accuracy', 'macro avg', 'weighted avg']:
                print(f"\n{label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name}: {metric_value:.4f}")
            else:
                print(f"\nClass {label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name}: {metric_value:.4f}")


In [8]:
param_grid = {
    'classifier__n_estimators': [ 200, 300 ],
    'classifier__max_depth': [20, 30],
    'classifier__min_samples_split': [2, 5],
    'classifier__max_features': ['sqrt'],
    'classifier__bootstrap': [True, False],
    'classifier__class_weight': [None, 'balanced']
}

result_folder = "../results/random_forest/"


In [ ]:
best_model, best_params, test_score = ML.grid_search_random_forest_eeg_2(
    X, y, param_grid, test_size=0.2, cv=5, scoring='accuracy'
)